# Using the Custom R Algorithm Container Image for Training & Inference with Amazon SageMaker Local Mode

In the previous recipe, we pushed the custom R container image to an **Amazon ECR** repository. In this recipe, we will perform the training and deployment steps in **Amazon SageMaker** using this custom container image. In the first chapter, we used the image URI of the container image of the built-in **Linear Learner**. In this chapter, we will use the image URI of the custom container image instead.

### How to do it...

In [ ]:
cmd <- function(bash_command) {
    output <- system(bash_command, intern=TRUE)
    last_line = ""
    
    for (line in output) { 
        cat(line)
        cat("\n")
        last_line = line 
    }
    
    return(last_line) 
}

In [ ]:
cmd("pip install 'sagemaker[local]' --upgrade")

In [ ]:
s3.bucket <- "<insert S3 bucket name here>"
s3.prefix <- "chapter01"

In [ ]:
training.s3_input_location <- paste0('s3://', s3.bucket, '/', s3.prefix, '/input/training_data.csv')
print(training.s3_input_location)

training.s3_output_location <- paste0('s3://', s3.bucket, '/', s3.prefix, '/output/custom/')
print(training.s3_output_location)

In [ ]:
library('reticulate')

sagemaker <- import('sagemaker')

In [ ]:
sagemaker[['__version__']]

In [ ]:
container <- "<insert container image URI>"

In [ ]:
role <- sagemaker$get_execution_role()

In [ ]:
LocalSession <- sagemaker$local$LocalSession

session <- LocalSession()
session$config <- list(local=list(local_code=TRUE))

In [ ]:
TrainingInput <- sagemaker$inputs$TrainingInput

sagemaker.train_input <- TrainingInput(training.s3_input_location, content_type="text/csv")

In [ ]:
Estimator <- sagemaker$estimator$Estimator

estimator <- Estimator(
    container,
    role, 
    instance_count=1L, 
    instance_type="local",
    output_path=training.s3_output_location,
    sagemaker_session=session)

In [ ]:
estimator$set_hyperparameters(a=1L, b=2L, c=3L)

In [ ]:
estimator$fit(list(train = sagemaker.train_input))

In [ ]:
estimator$model_data

In [ ]:
predictor <- estimator$deploy(
    initial_instance_count=1L,
    instance_type="local",
    endpoint_name="custom-local-r-endpoint"
)

In [ ]:
predictor$predict("42")

In [ ]:
predictor$delete_endpoint()

<img align="left" width="130" src="https://raw.githubusercontent.com/PacktPublishing/Amazon-SageMaker-Cookbook/master/Extra/cover-small-padded.png"/>

This notebook contains the code to help readers work through one of the recipes of the book [Machine Learning with Amazon SageMaker Cookbook: 80 proven recipes for data scientists and developers to perform ML experiments and deployments](https://www.amazon.com/Machine-Learning-Amazon-SageMaker-Cookbook/dp/1800567030)